# Dependencies 

# Importing Necessary Libraries

In [26]:
import cv2
import numpy as np

# Function Definitions

1.`define_color_ranges()` ---> This function defines the HSV color ranges for yellow, green, white, and orange balls. 
                             These ranges are used to create masks for detecting balls of specific colors.

In [27]:

def define_color_ranges():
    return {
        'yellow': (np.array([20, 100, 100]), np.array([30, 255, 255])),
        'green': (np.array([35, 50, 50]), np.array([85, 255, 255])),
        'white': (np.array([0, 0, 168]), np.array([172, 111, 255])),
        'orange': (np.array([5, 100, 100]), np.array([15, 255, 255]))
    }

2.`define_quadrants(frame, shift=300)` ---> This function divides the frame into four quadrants. 
                                          The quadrants are shifted to the right by 300 pixels to adjust for any misalignment in the video. 
                                          The function returns a dictionary with quadrant numbers as keys and their respective coordinate ranges as values.

In [28]:

def define_quadrants(frame, shift=300):
    height, width = frame.shape[:2]
    horizontal_line = height // 2
    vertical_line = width // 2 + shift 
    
    return {
        3: ((0, 0), (vertical_line, horizontal_line)),
        4: ((vertical_line, 0), (width, horizontal_line)),
        2: ((0, horizontal_line), (vertical_line, height)),
        1: ((vertical_line, horizontal_line), (width, height))
    }

3.`detect_balls(frame, color_ranges)` ---> This function detects balls of different colors in the frame.
                                         It converts the frame to HSV color space and creates masks for each color.
                                         Morphological operations are applied to remove noise.

In [29]:

def detect_balls(frame, color_ranges):
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    balls = {}
    for color, (lower, upper) in color_ranges.items():
        mask = cv2.inRange(hsv, lower, upper)
        # Morphological operations to remove noise and fill gaps
        mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, np.ones((7, 7), np.uint8))
        mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, np.ones((7, 7), np.uint8))
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        for contour in contours:
            if cv2.contourArea(contour) > 100:
                x, y, w, h = cv2.boundingRect(contour)
                center = (x + w // 2, y + h // 2)
                balls[color] = center
    return balls

4.`get_quadrant(point, quadrants)`---> This function determines which quadrant a given point (ball center) belongs to by comparing the point's coordinates with the quadrant boundaries.

In [30]:

def get_quadrant(point, quadrants):
    x, y = point
    for quadrant, ((x1, y1), (x2, y2)) in quadrants.items():
        if x1 <= x < x2 and y1 <= y < y2:
            return quadrant
    return None

5.`format_time(seconds)`---> This function formats a given time in seconds into a string in the format HH:MM:SS.sss.

In [31]:
def format_time(seconds):
    hours, remainder = divmod(seconds, 3600)
    minutes, seconds = divmod(remainder, 60)
    return f"{int(hours):02d}:{int(minutes):02d}:{seconds:06.3f}"


6.`process_video(input_path, output_video_path, output_text_path)` ---> 

In [32]:
def process_video(input_path, output_video_path, output_text_path):
    cap = cv2.VideoCapture(input_path)
    if not cap.isOpened():
        print(f"Error: Unable to open video file {input_path}")
        return

    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

    color_ranges = define_color_ranges()
    quadrants = None
    events = []
    ball_positions = {}
    frame_number = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame_number += 1
        timestamp = frame_number / fps

        if quadrants is None:
            quadrants = define_quadrants(frame)

        balls = detect_balls(frame, color_ranges)

        for color, center in balls.items():
            cv2.circle(frame, center, 5, (0, 255, 0), -1)
            current_quadrant = get_quadrant(center, quadrants)

            if color not in ball_positions:
                ball_positions[color] = current_quadrant
                events.append((timestamp, current_quadrant, color, 'Entry'))
            elif ball_positions[color] != current_quadrant:
                events.append((timestamp, ball_positions[color], color, 'Exit'))
                events.append((timestamp, current_quadrant, color, 'Entry'))
                ball_positions[color] = current_quadrant

        # Draw quadrant grid
        for q, ((x1, y1), (x2, y2)) in quadrants.items():
            cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
            cv2.putText(frame, f"Q{q}", (x1 + 10, y1 + 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)

        # Add overlay text for events and timestamp
        recent_events = [e for e in events if abs(e[0] - timestamp) < 1.0]
        for i, event in enumerate(recent_events[-4:]):
            event_text = f"{event[2]} ball {event[3]} Q{event[1]} at {format_time(event[0])}"
            cv2.putText(frame, event_text, (10, 60 + i*30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

        # Display the timestamp at the top of the frame
        cv2.putText(frame, format_time(timestamp), (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

        out.write(frame)

    cap.release()
    out.release()

    with open(output_text_path, 'w') as f:
        for event in events:
            f.write(f"{format_time(event[0])}, {event[1]}, {event[2]}, {event[3]}\n")

# Main Block

In [33]:



if __name__ == "__main__":
    input_video = 'AI Assignment video.mp4'
    output_video = 'output_video.mp4'
    output_text = 'output_events.txt'

    process_video(input_video, output_video, output_text)
